In [1]:
from json import dumps
import os, urllib.request
import pickle

import numpy as np
import pandas as pd
import sklearn
import datasets
import transformers
import torch
import tqdm.auto as tqdm

from projection_simplex_vectorized import projection_simplex
import postprocess

seed = 33
transformers.set_seed(seed)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")

In [2]:
model_name = "bert-base-uncased"
model_dir = "models/biasbios_squared_loss"

n_epochs = 3
batch_size = 32
lr = 2e-5
warmup_ratio = 0.1
weight_decay = 0.01
max_grad_norm = 1.0

split_ratio_for_postprocessing = 0.05

rng = np.random.default_rng(seed)
noise_fn = lambda shape: rng.laplace(loc=0.0, scale=0.2 / 28, size=shape)

## Download and load BiasBios dataset

In [3]:
label_names = [
    "accountant", "architect", "attorney", "chiropractor", "comedian",
    "composer", "dentist", "dietitian", "dj", "filmmaker", "interior_designer",
    "journalist", "model", "nurse", "painter", "paralegal", "pastor",
    "personal_trainer", "photographer", "physician", "poet", "professor",
    "psychologist", "rapper", "software_engineer", "surgeon", "teacher",
    "yoga_teacher"
]
n_labels = len(label_names)

group_names = ["female", "male"]
n_groups = len(group_names)

features = datasets.Features({
    "bio": datasets.Value("string"),
    "title": datasets.ClassLabel(names=label_names),
    "gender": datasets.ClassLabel(names=group_names),
})

train_path = "data/biasbios/train.pickle"
test_path = "data/biasbios/test.pickle"
dev_path = "data/biasbios/dev.pickle"
if any([not os.path.exists(p) for p in [train_path, test_path, dev_path]]):
  os.makedirs("data/biasbios", exist_ok=True)
  urllib.request.urlretrieve(
      "https://storage.googleapis.com/ai2i/nullspace/biasbios/train.pickle",
      train_path)
  urllib.request.urlretrieve(
      "https://storage.googleapis.com/ai2i/nullspace/biasbios/test.pickle",
      test_path)
  urllib.request.urlretrieve(
      "https://storage.googleapis.com/ai2i/nullspace/biasbios/dev.pickle",
      dev_path)

raw_dataset = {}
for split, path in zip(["train", "test", "dev"],
                       [train_path, test_path, dev_path]):
  rows = {k: [] for k in features}
  with open(path, "rb") as pickle_file:
    for row in pickle.load(pickle_file):
      rows["gender"].append("female" if row["g"] == "f" else "male")
      rows["title"].append(row["p"])
      rows["bio"].append(rows["gender"][-1] + ". " +
                         row["hard_text_untokenized"])
  raw_dataset[split] = datasets.Dataset.from_dict(rows, features=features)
raw_dataset = datasets.DatasetDict(raw_dataset)

print(raw_dataset["train"][0])

{'bio': 'female. She has been working with children in camp, community and school settings for the past 8 years. She believes in the importance of cultivating self-love and awareness in black children at a very young age and is excited to be apart of Black Lives Matter Toronto’s Freedom School!', 'title': 26, 'gender': 0}


In [4]:
# Compute and print dataset statistics

df = datasets.concatenate_datasets([d for d in raw_dataset.values()])
df = pd.DataFrame(np.stack(
    [np.array(group_names)[df["gender"]],
     np.array(label_names)[df["title"]]],
    axis=1),
                  columns=["Group", "Target"])
grouped = df.groupby(["Target", "Group"]).size().unstack()
n_labels = len(grouped.index)
n_groups = len(grouped.columns)
counts = grouped.sum(axis=0)
normalized = np.nan_to_num((grouped.to_numpy() / counts.to_numpy())).T
diff = np.abs(normalized[:, None, :] - normalized[None, :, :])
postprocessor = postprocess.PostProcessor()
postprocessor.fit(
    np.concatenate([np.eye(n_labels) for _ in range(n_groups)], axis=0),
    np.repeat(np.arange(n_groups), n_labels), normalized.flatten())
res = {
    "balanced_accuracy": {
        "perfect_postprocessed": (n_groups - postprocessor.score_) / n_groups
    },
    "dp_gap_linf_max": {
        "perfect_predictor": np.max(diff)
    },
    "dp_gap_l1_max": {
        "perfect_predictor": np.max(1 / 2 * np.sum(diff, axis=2))
    },
    "dp_gap_l1_avg": {
        "perfect_predictor":
            np.mean(1 / 2 * np.sum(diff, axis=2)[np.triu_indices(n_groups, 1)])
    },
}

display(pd.DataFrame(res))
display(grouped / counts)
display(pd.DataFrame(counts, columns=["Count"]).T)

,balanced_accuracy,dp_gap_linf_max,dp_gap_l1_max,dp_gap_l1_avg
perfect_postprocessed,0.884376,NaN,NaN,NaN
perfect_predictor,NaN,0.08644,0.231247,0.231247


Group,female,male
Target,,
accountant,0.011428,0.016898
architect,0.013168,0.036508
attorney,0.068610,0.095177
chiropractor,0.003789,0.009029
comedian,0.003251,0.010444
composer,0.005041,0.022156
dentist,0.028297,0.044132
dietitian,0.020258,0.001368
dj,0.001159,0.006029


Group,female,male
Count,182102,211321


### Tokenize BiasBios dataset

In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)


def tokenize_function(examples):
  tokenized_examples = tokenizer(examples["bio"],
                                 padding=False,
                                 max_length=tokenizer.model_max_length,
                                 truncation=True)
  tokenized_examples["labels"] = examples["title"]
  tokenized_examples["group_labels"] = examples["gender"]
  return tokenized_examples


tokenized_dataset = raw_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=raw_dataset["train"].column_names,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/256 [00:00<?, ?ba/s]

Running tokenizer on dataset:   0%|          | 0/99 [00:00<?, ?ba/s]

Running tokenizer on dataset:   0%|          | 0/40 [00:00<?, ?ba/s]

In [6]:
# Train data split
split_dataset = tokenized_dataset["train"].train_test_split(
    test_size=split_ratio_for_postprocessing, seed=seed)
train_dataset_predictor = split_dataset["train"]
train_dataset_postprocessor = split_dataset["test"]
eval_dataset = tokenized_dataset["test"]
dev_dataset = tokenized_dataset["dev"]

data_collator = transformers.DataCollatorWithPadding(tokenizer)
train_dataloader_predictor = torch.utils.data.DataLoader(
    train_dataset_predictor,
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size)
train_dataloader_postprocessor = torch.utils.data.DataLoader(
    train_dataset_postprocessor,
    collate_fn=data_collator,
    batch_size=batch_size)
eval_dataloader = torch.utils.data.DataLoader(eval_dataset,
                                              collate_fn=data_collator,
                                              batch_size=batch_size)
dev_dataloader = torch.utils.data.DataLoader(dev_dataset,
                                             collate_fn=data_collator,
                                             batch_size=batch_size)

## Load/train NLU model

In [7]:
model = None
is_finetuned = False
if os.path.exists(model_dir):
  model = transformers.AutoModelForSequenceClassification.from_pretrained(
      model_dir).to(device)
  is_finetuned = True
else:
  model = transformers.AutoModelForSequenceClassification.from_pretrained(
      model_name, num_labels=n_labels).to(device)

model_input_args = list(model.forward.__code__.co_varnames)

In [8]:
if not is_finetuned:
  no_decay = ["bias", "LayerNorm.weight"]
  optimizer_grouped_parameters = [
      {
          "params": [
              p for n, p in model.named_parameters()
              if not any(nd in n for nd in no_decay)
          ],
          "weight_decay": weight_decay,
      },
      {
          "params": [
              p for n, p in model.named_parameters()
              if any(nd in n for nd in no_decay)
          ],
          "weight_decay": 0.0
      },
  ]
  optimizer = transformers.AdamW(optimizer_grouped_parameters, lr=lr)
  lr_scheduler = transformers.get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=(warmup_ratio * n_epochs *
                        len(train_dataloader_predictor)),
      num_training_steps=n_epochs * len(train_dataloader_predictor))

  loss_fn = torch.nn.MSELoss()
  # loss_fn = torch.nn.CrossEntropyLoss()
  metric = datasets.load_metric("accuracy")

  for epoch in range(n_epochs):
    model.train()
    for batch in tqdm.tqdm(train_dataloader_predictor,
                           desc=f"Train epoch {epoch}"):
      batch = {
          k: v.to(device) for k, v in batch.items() if k in model_input_args
      }
      outputs = model(**batch)
      labels_one_hot = torch.nn.functional.one_hot(batch["labels"], n_labels)
      loss = loss_fn(outputs.logits, labels_one_hot.float())
      # loss = loss_fn(outputs.logits, batch["labels"])
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
    model.eval()

    for batch in tqdm.tqdm(dev_dataloader, desc=f"Dev epoch {epoch}"):
      batch = {
          k: v.to(device) for k, v in batch.items() if k in model_input_args
      }
      with torch.no_grad():
        outputs = model(**batch)
        probas = outputs.logits
        predictions = torch.argmax(probas, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])
    print(f"epoch {epoch}: {dumps(metric.compute(), indent=2)}")

  model.save_pretrained(model_dir)

## Post-process

In [9]:
def predict_fn(dataloader):
  model.eval()
  probas = []
  with torch.no_grad():
    for batch in tqdm.tqdm(dataloader, desc="Inference"):
      batch = {
          k: v.to(device) for k, v in batch.items() if k in model_input_args
      }
      outputs = model(**batch)
      probas.append(projection_simplex(outputs.logits.cpu().numpy(), axis=1))
      # probas.append(
      #     torch.nn.functional.softmax(outputs.logits, dim=-1).cpu().numpy())
  return np.concatenate(probas, axis=0)


postprocessor_path = os.path.join(model_dir, "postprocessor.pkl")
if os.path.exists(postprocessor_path):
  with open(postprocessor_path, "rb") as f:
    postprocessor = pickle.load(f)
else:
  postprocessor = postprocess.postprocess(
      predict_fn, train_dataloader_postprocessor,
      np.array(train_dataset_postprocessor["group_labels"]))
  with open(postprocessor_path, "wb") as f:
    pickle.dump(postprocessor, f)

res = postprocess.evaluate(predict_fn, postprocessor, eval_dataloader,
                           np.array(eval_dataset["labels"]),
                           np.array(eval_dataset["group_labels"]), n_labels,
                           n_groups)
display(pd.DataFrame(res))

Inference:   0%|          | 0/400 [00:00<?, ?it/s]

Inference:   0%|          | 0/3074 [00:00<?, ?it/s]

,accuracy,balanced_accuracy,dp_gap_linf_max,dp_gap_l1_max,dp_gap_l1_avg
predictor,0.857643,0.858601,0.105691,0.286086,0.286086
postprocessor,0.805031,0.806800,0.114265,0.135774,0.135774


In [10]:
postprocessor_path = os.path.join(model_dir, "postprocessor_smoothing.pkl")
if os.path.exists(postprocessor_path):
  with open(postprocessor_path, "rb") as f:
    postprocessor = pickle.load(f)
else:
  postprocessor = postprocess.postprocess(
      predict_fn,
      train_dataloader_postprocessor,
      np.array(train_dataset_postprocessor["group_labels"]),
      noise_fn=noise_fn,
      n_perturbations=10)
  with open(postprocessor_path, "wb") as f:
    pickle.dump(postprocessor, f)

res = postprocess.evaluate(predict_fn,
                           postprocessor,
                           eval_dataloader,
                           np.array(eval_dataset["labels"]),
                           np.array(eval_dataset["group_labels"]),
                           n_labels,
                           n_groups,
                           noise_fn=noise_fn,
                           n_perturbations=1000)
print("With Laplace smoothing:")
display(pd.DataFrame(res))

Inference:   0%|          | 0/400 [00:00<?, ?it/s]

Inference:   0%|          | 0/3074 [00:00<?, ?it/s]

With Laplace smoothing:


,accuracy,balanced_accuracy,dp_gap_linf_max,dp_gap_l1_max,dp_gap_l1_avg
predictor,0.857591,0.858546,0.105732,0.286233,0.286233
postprocessor,0.804342,0.806118,0.114086,0.128004,0.128004
